# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 25 2022 9:08AM,242607,10,0085902325,ISDIN Corporation,Released
1,Jul 25 2022 9:08AM,242607,10,0085902324,ISDIN Corporation,Released
2,Jul 25 2022 9:08AM,242607,10,0085902326,ISDIN Corporation,Released
3,Jul 25 2022 9:08AM,242607,10,0085902327,ISDIN Corporation,Released
4,Jul 25 2022 9:08AM,242607,10,0085902330,ISDIN Corporation,Released
5,Jul 25 2022 9:08AM,242607,10,0085902329,ISDIN Corporation,Released
6,Jul 25 2022 9:08AM,242607,10,0085902333,ISDIN Corporation,Released
7,Jul 25 2022 9:08AM,242607,10,0085902334,ISDIN Corporation,Released
8,Jul 25 2022 9:08AM,242607,10,0085902341,ISDIN Corporation,Released
9,Jul 25 2022 9:08AM,242607,10,0085902340,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,242603,Released,6
14,242604,Released,68
15,242605,Released,54
16,242606,Released,65
17,242607,Released,51


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242603,NaN,6.0
242604,NaN,68.0
242605,NaN,54.0
242606,NaN,65.0
242607,NaN,51.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242576,0.0,1.0
242579,0.0,1.0
242580,0.0,1.0
242583,0.0,1.0
242590,15.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242576,0,1
242579,0,1
242580,0,1
242583,0,1
242590,15,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242576,0,1
1,242579,0,1
2,242580,0,1
3,242583,0,1
4,242590,15,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242576,,1
1,242579,,1
2,242580,,1
3,242583,,1
4,242590,15,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 25 2022 9:08AM,242607,10,ISDIN Corporation
51,Jul 25 2022 9:08AM,242606,10,ISDIN Corporation
116,Jul 25 2022 9:06AM,242605,12,Hush Hush
170,Jul 25 2022 8:58AM,242604,10,ISDIN Corporation
238,Jul 25 2022 8:44AM,242603,10,"Snap Medical Industries, LLC"
244,Jul 25 2022 8:38AM,242602,10,ISDIN Corporation
260,Jul 25 2022 8:37AM,242601,10,"Amcyte Pharma, Inc."
266,Jul 25 2022 8:34AM,242600,10,Beach Products Inc
278,Jul 25 2022 8:30AM,242576,10,Template KITS
279,Jul 25 2022 8:30AM,242579,10,Eye Pharma Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 25 2022 9:08AM,242607,10,ISDIN Corporation,,51
1,Jul 25 2022 9:08AM,242606,10,ISDIN Corporation,,65
2,Jul 25 2022 9:06AM,242605,12,Hush Hush,,54
3,Jul 25 2022 8:58AM,242604,10,ISDIN Corporation,,68
4,Jul 25 2022 8:44AM,242603,10,"Snap Medical Industries, LLC",,6
5,Jul 25 2022 8:38AM,242602,10,ISDIN Corporation,,16
6,Jul 25 2022 8:37AM,242601,10,"Amcyte Pharma, Inc.",,6
7,Jul 25 2022 8:34AM,242600,10,Beach Products Inc,,12
8,Jul 25 2022 8:30AM,242576,10,Template KITS,,1
9,Jul 25 2022 8:30AM,242579,10,Eye Pharma Inc,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 25 2022 9:08AM,242607,10,ISDIN Corporation,51,
1,Jul 25 2022 9:08AM,242606,10,ISDIN Corporation,65,
2,Jul 25 2022 9:06AM,242605,12,Hush Hush,54,
3,Jul 25 2022 8:58AM,242604,10,ISDIN Corporation,68,
4,Jul 25 2022 8:44AM,242603,10,"Snap Medical Industries, LLC",6,
5,Jul 25 2022 8:38AM,242602,10,ISDIN Corporation,16,
6,Jul 25 2022 8:37AM,242601,10,"Amcyte Pharma, Inc.",6,
7,Jul 25 2022 8:34AM,242600,10,Beach Products Inc,12,
8,Jul 25 2022 8:30AM,242576,10,Template KITS,1,
9,Jul 25 2022 8:30AM,242579,10,Eye Pharma Inc,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 25 2022 9:08AM,242607,10,ISDIN Corporation,51,
1,Jul 25 2022 9:08AM,242606,10,ISDIN Corporation,65,
2,Jul 25 2022 9:06AM,242605,12,Hush Hush,54,
3,Jul 25 2022 8:58AM,242604,10,ISDIN Corporation,68,
4,Jul 25 2022 8:44AM,242603,10,"Snap Medical Industries, LLC",6,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 25 2022 9:08AM,242607,10,ISDIN Corporation,51.0,NaN
1,Jul 25 2022 9:08AM,242606,10,ISDIN Corporation,65.0,NaN
2,Jul 25 2022 9:06AM,242605,12,Hush Hush,54.0,NaN
3,Jul 25 2022 8:58AM,242604,10,ISDIN Corporation,68.0,NaN
4,Jul 25 2022 8:44AM,242603,10,"Snap Medical Industries, LLC",6.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 25 2022 9:08AM,242607,10,ISDIN Corporation,51.0,0.0
1,Jul 25 2022 9:08AM,242606,10,ISDIN Corporation,65.0,0.0
2,Jul 25 2022 9:06AM,242605,12,Hush Hush,54.0,0.0
3,Jul 25 2022 8:58AM,242604,10,ISDIN Corporation,68.0,0.0
4,Jul 25 2022 8:44AM,242603,10,"Snap Medical Industries, LLC",6.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2911138,229.0,0.0
12,242605,54.0,0.0
16,242595,0.0,1.0
19,727773,17.0,15.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2911138,229.0,0.0
1,12,242605,54.0,0.0
2,16,242595,0.0,1.0
3,19,727773,17.0,15.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,229.0,0.0
1,12,54.0,0.0
2,16,0.0,1.0
3,19,17.0,15.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,229.0
1,12,Released,54.0
2,16,Released,0.0
3,19,Released,17.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19
Status,,,,
Executing,0.0,0.0,1.0,15.0
Released,229.0,54.0,0.0,17.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19
0,Executing,0.0,0.0,1.0,15.0
1,Released,229.0,54.0,0.0,17.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19
0,Executing,0.0,0.0,1.0,15.0
1,Released,229.0,54.0,0.0,17.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()